In [1]:
%gui qt

import logging

from pyqtgraph.flowchart import Flowchart
from pyqtgraph import Qt
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib
from pyqtgraph.dockarea import Dock, DockArea

from plottr.data import datadict as dd
from plottr.data.datadict import DataDict, MeshgridDataDict
from plottr import log as plottrlog
from plottr.node.node import Node
from plottr.node.data_selector import DataSelector
from plottr.node.dim_reducer import XYAxesSelector
from plottr.plot.mpl import PlotNode, AutoPlot

import data

In [2]:
Node.raiseExceptions = True
DataSelector.raiseExceptions = True
XYAxesSelector.raiseExceptions = True
PlotNode.raiseExceptions = True

nodelib = fclib.NodeLibrary()
nodelib.addNodeType(DataSelector, [('Basic')])
nodelib.addNodeType(XYAxesSelector, [('Basic')])
nodelib.addNodeType(PlotNode, [('Plot')])

### Setting up the flowchart

# library
fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib

# flowchart for this example
datasel = fc.createNode('DataSelector')
xysel = fc.createNode('XYAxesSelector')
plot = fc.createNode('Plot')

fc.connectTerminals(fc['dataIn'], datasel['dataIn'])
fc.connectTerminals(datasel['dataOut'], xysel['dataIn'])
fc.connectTerminals(xysel['dataOut'], fc['dataOut'])
fc.connectTerminals(xysel['dataOut'], plot['dataIn'])

plotWidget = AutoPlot()
plot.setPlotWidget(plotWidget)


### Setting up the GUI
win = QtGui.QDialog()
layout = QtGui.QVBoxLayout()
area = DockArea()
layout.addWidget(area)
# win.setCentralWidget(area)
win.setLayout(layout)

dataselDock = Dock('Data Selector')
dataselDock.addWidget(datasel.ui)
area.addDock(dataselDock)

# xy selector
xyselDock = Dock('XY Axes Selector')
xyselDock.addWidget(xysel.ui)
area.addDock(xyselDock, 'bottom', dataselDock)

# logger
logDock = Dock('Log')
logDock.addWidget(plottrlog.setupLogging(makeDialog=False))
area.addDock(logDock, 'bottom', xyselDock)

# plot
plotDock = Dock('Plot')
plotDock.addWidget(plotWidget)
area.addDock(plotDock, 'right')

win.show()

### Logging
logger = logging.getLogger('plottr.interactive')
plottrlog.LEVEL = logging.DEBUG

In [3]:
d = data.three_compatible_3d_sets(21, 21, 21)
d = dd.datadict_to_meshgrid(d)
fc.setInput(dataIn=d)

In [4]:
# datasel.selectedData = ['data', 'more_data']

In [5]:
# xysel.xyAxes = ('x', 'y')

In [6]:
# fc.output()

In [7]:
# xysel.reductions, xysel.xyAxes

In [8]:
# xysel.outputValues()

In [9]:
# xysel.ui._getReductions()